In [1]:
import torch
from model import ECG_ResNeXt

# Function to process the ECG data before feeding it into the model
def get_inputs(device, ecg_batch, apply="non_zero", signal_crop_len=2560):
    # Process ECG data
    if ecg_batch.shape[1] > ecg_batch.shape[2]:
        ecg_batch = ecg_batch.permute(0, 2, 1)
    B, n_leads, signal_len = ecg_batch.shape

    if apply == "non_zero":
        transformed_ecg = torch.zeros(B, n_leads, signal_crop_len)
        for b in range(B):
            # Infer signal_non_zero_start dynamically for each ECG
            start = torch.nonzero(ecg_batch[b, :, :], as_tuple=False)
            if start.nelement() == 0:
                start = 0
            else:
                start = start[0, 1].item()
            
            end = start + signal_crop_len
            # Adjust start and end if end exceeds signal_len
            if end > signal_len:
                end = signal_len
                start = end - signal_crop_len
                if start < 0:
                    start = 0

            for l in range(n_leads):
                transformed_ecg[b, l, :] = ecg_batch[b, l, start:end]
    else:
        transformed_ecg = ecg_batch

    return transformed_ecg.to(device)



Input shape: torch.Size([16, 12, 2560])
Output shape: torch.Size([16, 2])


In [ ]:
# Initialize model parameters
n_classes = 2
num_blocks = 3
channels = [64, 128, 192, 256]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize the ECG_ResNeXt model and move it to the device
model = ECG_ResNeXt(n_classes=n_classes, num_blocks=num_blocks, channels=channels).to(device)

# Generate a random ECG signal (batch size of 16, 12 leads, signal length of 4096)
batch_size = 16
signal_len = 4096
n_leads = 12

# Create a random ECG signal
ecg_batch = torch.randn(batch_size, n_leads, signal_len).to(device)

# Process the ECG batch using the get_inputs function
processed_ecg = get_inputs(device, ecg_batch, signal_crop_len=2560)

# Perform a forward pass
logits = model(processed_ecg)

# Print the input shape and output shape
print(f"Input shape: {processed_ecg.shape}")
print(f"Output shape: {logits.shape}")

In [24]:
import pandas as pd
import numpy as np

#CARREGAR OS DADOS
caminho_arquivo = "../Projeto/Database/exams.csv"
dados = pd.read_csv(caminho_arquivo)
arquivos_usados = ["exams_part2.hdf5", "exams_part3.hdf5", "exams_par4.hdf5", "exams_part5.hdf5",
                   "exams_part6.hdf5", "exams_part7.hdf5", "exams_par8.hdf5", "exams_part9.hdf5",
                   "exams_part10.hdf5", "exams_part11.hdf5", "exams_part12.hdf5", "exams_part13.hdf5", 
                   "exams_part14.hdf5", "exams_part15.hdf5", "exams_part16.hdf5", "exams_part17.hdf5"]

ecg_normal_linhas = dados.index[(dados.iloc[:, 14].isin(arquivos_usados))  & 
                                (dados.iloc[:, 14].isin(arquivos_usados)) & 
                                (dados.iloc[:, 4] == False) & 
                                (dados.iloc[:, 5] == False) & 
                                (dados.iloc[:, 6] == False) & 
                                (dados.iloc[:, 7] == False) & 
                                (dados.iloc[:, 8] == True) & 
                                (dados.iloc[:, 9] == False)]

ecg_doente_linhas = dados.index[(dados.iloc[:, 14].isin(arquivos_usados)) & 
                                (dados.iloc[:, 4] == True) & 
                                (dados.iloc[:, 5] == False) & 
                                (dados.iloc[:, 6] == False) & 
                                (dados.iloc[:, 7] == False) & 
                                (dados.iloc[:, 8] == False) & 
                                (dados.iloc[:, 9] == False)]

print("Número de linhas ecg_normal_linhas:", len(ecg_normal_linhas))
print("Número de linhas ecg_doente_linhas:", len(ecg_doente_linhas))

caminho_interferencias = "../Projeto/Database/resultados_interferencia.csv"
interferencias = pd.read_csv(caminho_interferencias)
interferencias_ids = interferencias['exam_id'].tolist()

ecg_normal_linhas = dados.index[~dados['exam_id'].isin(interferencias_ids) &
                                (dados.iloc[:, 14].isin(arquivos_usados))  & 
                                (dados.iloc[:, 14].isin(arquivos_usados)) & 
                                (dados.iloc[:, 4] == False) & 
                                (dados.iloc[:, 5] == False) & 
                                (dados.iloc[:, 6] == False) & 
                                (dados.iloc[:, 7] == False) & 
                                (dados.iloc[:, 8] == True) & 
                                (dados.iloc[:, 9] == False)]

ecg_doente_linhas = dados.index[~dados['exam_id'].isin(interferencias_ids) &
                                (dados.iloc[:, 14].isin(arquivos_usados)) & 
                                (dados.iloc[:, 4] == True) & 
                                (dados.iloc[:, 5] == False) & 
                                (dados.iloc[:, 6] == False) & 
                                (dados.iloc[:, 7] == False) & 
                                (dados.iloc[:, 8] == False) & 
                                (dados.iloc[:, 9] == False)]

print("Tirando Interferência:")
print("Número de linhas ecg_normal_linhas:", len(ecg_normal_linhas))
print("Número de linhas ecg_doente_linhas:", len(ecg_doente_linhas))

ecg_doente_id = dados.iloc[ecg_doente_linhas, 0].tolist()
ecg_normal_id = dados.iloc[ecg_normal_linhas, 0].tolist()

#ecg_doente = ecg_doente_id[]
ecg_normal = ecg_normal_id[:500]

ids_ecgs= ecg_doente_id[:500] + ecg_normal
print("Número de ecgs pra clusterizar:", len(ids_ecgs))

y_doente = [1] * len(ecg_doente_id[:500])  # Rótulo 1 para os doentes
y_normal = [0] * len(ecg_normal[:500])     # Rótulo 0 para os normais

# Concatena os labels
Y = y_doente + y_normal

print("Número de ecgs pra clusterizar:", len(Y))


Número de linhas ecg_normal_linhas: 5352
Número de linhas ecg_doente_linhas: 3219
Tirando Interferência:
Número de linhas ecg_normal_linhas: 5259
Número de linhas ecg_doente_linhas: 3146
Número de ecgs pra clusterizar: 1000
Número de ecgs pra clusterizar: 1000


In [25]:

import h5py
#X

arquivos_hdf5 = ["../Projeto/Database/filtered_exams_2_3.hdf5",  "../Projeto/Database/filtered_exams_4_5.hdf5",
                 "../Projeto/Database/filtered_exams_6_7.hdf5",  "../Projeto/Database/filtered_exams_8_9.hdf5",
                 "../Projeto/Database/filtered_exams_10_11.hdf5",  "../Projeto/Database/filtered_exams_12_13.hdf5",
                 "../Projeto/Database/filtered_exams_14_15.hdf5", "../Projeto/Database/filtered_exams_16_17.hdf5"]

def get_ecg_data(file_path, exam_id):
    with h5py.File(file_path, 'r') as f:
        # Obter os IDs dos exames
        exam_ids = np.array(f['exam_id'])

        # Encontrar o índice correspondente ao exam_id de interesse
        exam_index = np.where(exam_ids == exam_id)[0]

        if len(exam_index) == 0:
            raise ValueError("Exam ID não encontrado.")
        else:
            exam_index = exam_index[0]
            # Acessar os tracings de ECG correspondentes ao exam_index
            exam_tracings = f['tracings'][exam_index]
            # Preencher tracings nulos com epsilon
            return exam_tracings

exam_ids_to_cluster = ids_ecgs  # Substitua pelos IDs reais dos exames

# Lista para armazenar todos os tracings de ECG
all_tracings = []

# Obter os tracings de ECG para cada exam_id e armazenar na lista
for exam_id in exam_ids_to_cluster:
    found = False  # Sinalizador para verificar se o exame foi encontrado em algum arquivo
    for arquivo in arquivos_hdf5:
        try:
            tracings = get_ecg_data(arquivo, exam_id)
            if tracings is not None:
                tracing_transposto = np.array(tracings).T
                all_tracings.append(tracing_transposto)
                found = True  # Sinalizador para indicar que o exame foi encontrado
                break  # Se encontrou, não precisa continuar buscando nos outros arquivos
        except ValueError as e:
            i = 0
        except Exception as e:
            i = 0
    
    if not found:
        print(f"Erro: exame ID {exam_id} não encontrado em nenhum dos arquivos.")

# Verifique o tamanho da lista all_tracings para garantir que os dados foram coletados corretamente
print("Número de ecgs que eram pra ser processados:", len(ids_ecgs))
print(f"Número total de traçados processados: {len(all_tracings)}")

# X será um array com um único array dentro, contendo todos os números do tracings.T
X = np.array(all_tracings)




Número de ecgs que eram pra ser processados: 1000
Número total de traçados processados: 1000


In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, TensorDataset
from sklearn.metrics import accuracy_score

# Initialize model parameters
n_classes = 2
num_blocks = 3
channels = [64, 128, 192, 256]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize the ECG_ResNeXt model and move it to the device
model = ECG_ResNeXt(n_classes=n_classes, num_blocks=num_blocks, channels=channels).to(device)

# Create random ECG signals and labels
batch_size = 1000
signal_len = 4096
n_leads = 12

X = torch.tensor(X, dtype=torch.float32)  # Converte X em um tensor de ponto flutuante
Y = torch.tensor(Y, dtype=torch.long)     # Converte Y em um tensor de labels inteiros



X = get_inputs(device, X, signal_crop_len=2560)
# Split dataset into training (80%) and testing (20%)
train_size = int(0.9 * len(X))
test_size = len(X) - train_size
train_dataset, test_dataset = random_split(TensorDataset(X, Y), [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Simplified training loop
n_epochs = 5  # Reduzi o número de épocas para agilizar o treinamento
for epoch in range(n_epochs):
    model.train()  # Modo de treinamento
    train_loss = 0.0
    for ecg_batch, labels in train_loader:
        ecg_batch, labels = ecg_batch.to(device), labels.to(device)

        # Forward pass
        outputs = model(ecg_batch)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    print(f"Epoch [{epoch+1}/{n_epochs}], Loss: {train_loss/len(train_loader):.4f}")

# Simplified evaluation (no detailed metrics, just accuracy)
model.eval()  # Modo de avaliação
all_preds = []
all_labels = []
with torch.no_grad():  # Desativa cálculo de gradientes para acelerar
    for ecg_batch, labels in test_loader:
        ecg_batch, labels = ecg_batch.to(device), labels.to(device)
        outputs = model(ecg_batch)
        _, preds = torch.max(outputs, 1)
        all_preds.append(preds.cpu())
        all_labels.append(labels.cpu())

all_preds = torch.cat(all_preds)
all_labels = torch.cat(all_labels)
accuracy = accuracy_score(all_labels, all_preds)
print(f"Test Accuracy: {accuracy:.4f}")


Epoch [1/5], Loss: 520.8836
Epoch [2/5], Loss: 394.6088
Epoch [3/5], Loss: 761.9548
Epoch [4/5], Loss: 212.4404
Epoch [5/5], Loss: 259.2702
Test Accuracy: 0.6100
